In [1]:
from kaggle_secrets import UserSecretsClient

secret_OpenAI = UserSecretsClient().get_secret("OpenAI_KEY")
secret_Langchain = UserSecretsClient().get_secret("llmLangChain_test")

In [ ]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain


In [6]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = secret_Langchain
os.environ['OPENAI_API_KEY'] = secret_OpenAI


In [18]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader,PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


In [20]:
# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# import os

# pdf=[]
# for dirname, _, filenames in os.walk('/kaggle/input/sysinfo'):
#     for filename in filenames:
#         pdf_path=os.path.join(dirname, filename)
#         loader_pdf = PyPDFLoader(pdf_path)
#         pdf.extend(loader_pdf.load())

# #         pdf.append(loader)
        
        



In [69]:
loader_pdf = PyPDFLoader("/kaggle/input/eonsight/eOnsight - Stage 2024 - OCR.pdf")
pdf_eOnsight =loader_pdf.load()


doc

In [70]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(pdf_eOnsight)

In [73]:
# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())


retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [74]:
result = retriever.invoke("L’objectif d’eOnsight")

result


[Document(page_content='cas d’intégration via LLM.  \nL’objectif d’eOnsight est de croiser les données pouvant être ingéré es de ces rapports d’inspection \navec des données satellites ou autres données disponibles sur Internet.  \n \nPréparation', metadata={'page': 0, 'source': '/kaggle/input/eonsight/eOnsight - Stage 2024 - OCR.pdf'})]

In [75]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'))])

In [76]:
chain = prompt | llm

In [77]:
chain.invoke({"context":docs,"question":"L’objectif d’eOnsight"})

AIMessage(content='L’objectif d’eOnsight est de croiser les données pouvant être ingérées de rapports d’inspection avec des données satellites ou autres données disponibles sur Internet.', response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 655, 'total_tokens': 693}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6019bd56-b69a-4707-a978-806ff3c011e3-0')

# Using json data 

In [78]:
import json
from langchain_text_splitters import RecursiveJsonSplitter

In [79]:
json_file_path = "/kaggle/input/json-file/json_example.json"

# Read the JSON file and load its contents into a variable
with open(json_file_path, 'r') as json_file:
    json_data  = json.load(json_file)

# Print the contents of the variable
print(json_data )

{"('EQUIPEMENTS', 'SUR QUVRAGE', 'SUR QUVRAGE')": {'0': 'SUR QUVRAGE', '1': 'Chaussée', '2': 'Trottoirs et bordures', '3': 'Dispositifs de retenue', '4': 'Corniches', '5': "Dispositifs d'évacuation des eaux", '6': 'Joints de chaussée et de trottoirs', '7': 'Autres équipements sur ouvrage'}, "('EQUIPEMENTS', 'SUBDI', 'classe')": {'0': 'classe', '1': '2', '2': '1', '3': '3', '4': '4', '5': '2', '6': None, '7': 'S.0'}, "('EQUIPEMENTS', 'SUBDI', nan)": {'0': None, '1': None, '2': None, '3': None, '4': None, '5': None, '6': None, '7': None}, "('EQUIPEMENTS', 'CDOA', 'classe')": {'0': 'classe', '1': None, '2': None, '3': None, '4': None, '5': None, '6': None, '7': None}, "('EQUIPEMENTS', 'CDOA', '5')": {'0': '5', '1': None, '2': None, '3': None, '4': None, '5': None, '6': None, '7': None}}


In [82]:
splitter = RecursiveJsonSplitter(max_chunk_size=300)

json_chunks = splitter .split_json(json_data=json_data)

In [95]:
# The splitter can also output documents
docs = splitter.create_documents(texts=[json_data])

# or a list of strings
texts = splitter.split_text(json_data=json_data)

print(texts[0])
print(texts[1])

{"('EQUIPEMENTS', 'SUR QUVRAGE', 'SUR QUVRAGE')": {"0": "SUR QUVRAGE", "1": "Chauss\u00e9e", "2": "Trottoirs et bordures", "3": "Dispositifs de retenue", "4": "Corniches", "5": "Dispositifs d'\u00e9vacuation des eaux", "6": "Joints de chauss\u00e9e et de trottoirs"}}
{"('EQUIPEMENTS', 'SUR QUVRAGE', 'SUR QUVRAGE')": {"7": "Autres \u00e9quipements sur ouvrage"}, "('EQUIPEMENTS', 'SUBDI', 'classe')": {"0": "classe", "1": "2", "2": "1", "3": "3", "4": "4", "5": "2", "6": null, "7": "S.0"}}


In [102]:
docs[3]

Document(page_content='{"(\'EQUIPEMENTS\', \'CDOA\', \'5\')": {"0": "5", "1": null, "2": null, "3": null, "4": null, "5": null, "6": null, "7": null}}')

In [103]:
# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=docs, 
                                    embedding=OpenAIEmbeddings())


retrievers = vectorstore.as_retriever(search_kwargs={"k": 2})

In [105]:
result = retrievers.invoke("Dispositifs")

result[0]


Document(page_content='de numérisation de documents sous formats hétéroclites, maîtriser les outils et techniques d’OCR est \nessentiel. C’est la première brique nécessaire à l’élaboration de pipeline d’analyse IA comme dans le \ncas d’intégration via LLM.', metadata={'page': 0, 'source': '/kaggle/input/eonsight/eOnsight - Stage 2024 - OCR.pdf'})